In [1]:
import numpy as np
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision
import torch

def slice_output(output: dict, confidence_threshold: float = 0.7) -> dict:
    """
    this method is responsible for validating models output w.r.t confidence_threshold defined above.
    It accepts an output dictionary from model, namely {'boxes':[], 'labels':[], 'scores':[]}
    It returns a dictionary sliced to items with score above confidence_threshold
    """

    num_valid_elements = np.sum(np.array(output['scores']) >= confidence_threshold)
    # temporary option to make sure, that it returns at least one element, although it should probably be fixed, 
    # should there be any frames where there is no starfish
    if num_valid_elements == 0:
        num_valid_elements = 1
    res = {}
    for key, value in output.items():
        res[key] = value[:num_valid_elements]
    return res

def format_for_submission(output:dict) -> str:
    """
    this method is responsible for formatting output so that it fits a submission format
    """
    output = slice_output(output)
    res_str = ''
    for bbox, score in zip(output['boxes'], output['scores']):
        # print(bbox, score)
        w = bbox[2] - bbox[0]
        h = bbox[1] - bbox[3]
        x = bbox[0]
        y = bbox[1]
        res_str += '{} {} '.format(score, ''.join(map(str, [x, y, w, h])))
    return res_str

In [2]:
num_classes = 2  # starfish and not starfish I guess

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained_backbone=False)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# load trained model
model.load_state_dict(torch.load('/kaggle/input/modelsmall/model2.pt', map_location=torch.device('cpu')))


<All keys matched successfully>

In [3]:
import sys
sys.path.append('/kaggle/input/tensorflow-great-barrier-reef')
import greatbarrierreef
env = greatbarrierreef.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
with torch.no_grad():

    for (pixel_array, sample_prediction_df) in iter_test:

        img = torch.from_numpy(pixel_array)
        img = torch.permute(img, (2, 0, 1))
        min_image = img.min()
        max_image = img.max()
        # normalize image to 0-1 - required by torchvision
        img -= min_image
        img = torch.FloatTensor(img/max_image)
        img = torch.unsqueeze(img, 0) # add dummy batch dimension
        model.eval()
        output = model(img)
        result = format_for_submission(output[0])
#         print(result)
        sample_prediction_df['annotations'] = result  # make your predictions here
        env.predict(sample_prediction_df)   # register your predictions
    sample_prediction_df.to_csv('/kaggle/working/submission.csv')

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
